In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
cd "drive/My Drive/Deep-Learning/Research/Beijing Housing Price"

[Errno 2] No such file or directory: 'drive/My Drive/Deep-Learning/Research/Beijing Housing Price'
/content/drive/My Drive/Deep-Learning/Research/Beijing Housing Price


In [0]:
#%% Import neccesary frameworks and libs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import model_utils
from sklearn.model_selection import train_test_split

In [0]:
#%% Import Dataset
csv_file = "FloorFixed.csv"
df = pd.read_csv(csv_file, low_memory= False, encoding= "latin1")

In [0]:
df.head()

,Unnamed: 0,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,floorType,floorHeight
0,0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2,1,1,1,é« 26,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0,High,26.0
1,1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2,2,1,2,é« 22,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0,High,22.0
2,2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3,2,1,3,ä¸­ 4,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0,Middle,4.0
3,3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3,1,1,1,åº 21,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0,Bottom,21.0
4,4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2,1,1,1,ä¸­ 6,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0,Middle,6.0


In [0]:
df.isnull().sum()

Unnamed: 0                  0
url                         0
id                          0
Lng                         0
Lat                         0
Cid                         0
tradeTime                   0
DOM                    157977
followers                   0
totalPrice                  0
price                       0
square                      0
livingRoom                  0
drawingRoom                 0
kitchen                     0
bathRoom                    0
floor                       0
buildingType             2021
constructionTime            0
renovationCondition         0
buildingStructure           0
ladderRatio                 0
elevator                   32
fiveYearsProperty          32
subway                     32
district                    0
communityAverage          463
floorType                   0
floorHeight                32
dtype: int64

In [0]:
#%% Drop unnecessary attributes
df = df.drop(columns= ["DOM", "kitchen", "drawingRoom", "bathRoom", "id", "Cid", "totalPrice", "floor"])

#df["floor"] = df["floor"].str[-2:] # Remove chinese character
df = df[(df["renovationCondition"] == 1) | (df["renovationCondition"] == 2) |(df["renovationCondition"] == 3) |(df["renovationCondition"] == 4)]

df = df[(df["buildingType"] == 1) | (df["buildingType"] == 2) | (df["buildingType"] == 3) | (df["buildingType"] == 4)]

df = df[df["buildingStructure"] != 0]

df = df[df["livingRoom"] != "#NAME?"] # Remove invalid rows contains NAME?

df = df[(df["constructionTime"] != "1") & (df["constructionTime"] != "0") & (df["constructionTime"] != "æ\x9cªç\x9f¥")] # Remove invalid rows contain invalid Construction Time

df = df.dropna(how="any") # Remove invalid rows contain NaN

df = df[(df["livingRoom"] == "1") | (df["livingRoom"] == "2") | (df["livingRoom"] == "3") | (df["livingRoom"] == "4")] # Remove invalid rows contain too many livingRoom

df = df.assign(distance= ((df["Lng"] -116.4074)**2 + (df["Lat"] - 39.9042)**2)**0.5) # Distance from center

df = df.assign(age= (2019 - df["constructionTime"].astype("int"))) # Age of building

#df = df.drop(columns=["constructionTime"])

df = df[df["price"].astype("float") >= 500] # Drop invalid housing price

df = df[df["square"].astype("float") >= 20] # Drop invalid area

df = df[df["ladderRatio"].astype("float") <= 5] # Drop invalid ladderRatio

df["tradeTime"] = df["tradeTime"].str[:4].astype("int")

#df = model_utils.handle_missing_values(df, cat_fillna= ["buildingType", "elevator", "fiveYearsProperty", "subway"], num_fillna= ["communityAverage"])

#df = df.dropna(how="any")

df = df.reset_index(drop= True)


In [0]:
#%% Arrange Columns
df = df[['url', 'price', 'Lng', 'Lat', 'district', 'distance', 'age', 'square', 
          "communityAverage","followers", 'tradeTime', 'constructionTime', 
          'livingRoom', 'floorType', 'floorHeight',
          'buildingType', 'renovationCondition', 'buildingStructure',
          'ladderRatio', 'elevator', 'fiveYearsProperty', 'subway']]

In [0]:
#%% Convert every numeric column to numeric data type
df["livingRoom"] = df["livingRoom"].astype("int")
df["floorHeight"] = df["floorHeight"].astype("int")
df["buildingType"] = df["buildingType"].astype("int")
df["elevator"] = df["elevator"].astype("int")
df["fiveYearsProperty"] = df["elevator"].astype("int")
df["subway"] = df["subway"].astype("int")
df["constructionTime"] = df["constructionTime"].astype("int")


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296150 entries, 0 to 296149
Data columns (total 22 columns):
url                    296150 non-null object
price                  296150 non-null int64
Lng                    296150 non-null float64
Lat                    296150 non-null float64
district               296150 non-null int64
distance               296150 non-null float64
age                    296150 non-null int64
square                 296150 non-null float64
communityAverage       296150 non-null float64
followers              296150 non-null int64
tradeTime              296150 non-null int64
constructionTime       296150 non-null int64
livingRoom             296150 non-null int64
floorType              296150 non-null object
floorHeight            296150 non-null int64
buildingType           296150 non-null int64
renovationCondition    296150 non-null int64
buildingStructure      296150 non-null int64
ladderRatio            296150 non-null float64
elevator            

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296150 entries, 0 to 296149
Data columns (total 22 columns):
url                    296150 non-null object
price                  296150 non-null int64
Lng                    296150 non-null float64
Lat                    296150 non-null float64
district               296150 non-null int64
distance               296150 non-null float64
age                    296150 non-null int64
square                 296150 non-null float64
communityAverage       296150 non-null float64
followers              296150 non-null int64
tradeTime              296150 non-null int64
constructionTime       296150 non-null int64
livingRoom             296150 non-null int64
floorType              296150 non-null object
floorHeight            296150 non-null int64
buildingType           296150 non-null int64
renovationCondition    296150 non-null int64
buildingStructure      296150 non-null int64
ladderRatio            296150 non-null float64
elevator            

In [0]:
#%% Detect Outliers
#outliers = utils.detect_outliers(df, 1, ["livingRoom", "ladderRatio", "age", "square", "price"])
#df.loc[outliers].count()
#df = df.drop(outliers, axis= 0).reset_index(drop= True)

outliers = []

outliers.extend(model_utils.detect_outliers(df[df["elevator"] == 0], 1, ["floorHeight", "distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["elevator"] == 1], 1, ["floorHeight", "distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["fiveYearsProperty"] == 0], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["fiveYearsProperty"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["subway"] == 0], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["subway"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 2], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 3], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 4], 0, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 2], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 3], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 4], 1, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 2], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 3], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 4], 1, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 1], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 2], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 3], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 4], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 5], 1, ["distance", "age", "square", "price", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 6], 1, ["distance", "age", "square", "price", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["elevator"] == 0], 0, ["floorHeight", "age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["elevator"] == 1], 0, ["floorHeight", "age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["fiveYearsProperty"] == 0], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["fiveYearsProperty"] == 1], 0, ["age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["subway"] == 0], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["subway"] == 1], 0, ["age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 1], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 2], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 3], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["livingRoom"] == 4], 0, ["age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 1], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 2], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 3], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["renovationCondition"] == 4], 0, ["age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 1], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 2], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 3], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingType"] == 4], 0, ["age", "square", "communityAverage"]))

outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 1], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 2], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 3], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 4], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 5], 0, ["age", "square", "communityAverage"]))
outliers.extend(model_utils.detect_outliers(df[df["buildingStructure"] == 6], 0, ["age", "square", "communityAverage"]))

outliers = list(set(outliers))
df.loc[outliers]
df = df.drop(outliers, axis= 0).reset_index(drop= True)

In [0]:
print(df["price"].min(), df["price"].max())
print(df["square"].min(), df["square"].max())
print(df["livingRoom"].min(), df["livingRoom"].max())
print(df["age"].min(), df["age"].max())
print(df["ladderRatio"].min(), df["ladderRatio"].max())
print(df[df["livingRoom"] == 4].shape)

572 149893
20.0 161.51
1 4
3 45
0.019 5.0
(989, 22)


In [0]:
df.to_csv("Final_Dataset.csv", index= False)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231962 entries, 0 to 231961
Data columns (total 22 columns):
url                    231962 non-null object
price                  231962 non-null int64
Lng                    231962 non-null float64
Lat                    231962 non-null float64
district               231962 non-null int64
distance               231962 non-null float64
age                    231962 non-null int64
square                 231962 non-null float64
communityAverage       231962 non-null float64
followers              231962 non-null int64
tradeTime              231962 non-null int64
constructionTime       231962 non-null int64
livingRoom             231962 non-null int64
floorType              231962 non-null object
floorHeight            231962 non-null int64
buildingType           231962 non-null int64
renovationCondition    231962 non-null int64
buildingStructure      231962 non-null int64
ladderRatio            231962 non-null float64
elevator            

In [0]:
df

,url,price,Lng,Lat,district,distance,age,square,communityAverage,followers,tradeTime,constructionTime,livingRoom,floorType,floorHeight,buildingType,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway
0,https://bj.lianjia.com/chengjiao/101084782030....,31680,116.475489,40.019520,7,0.133921,14,131.00,56021.0,106,2016,2005,2,High,26,1,3,6,0.217,1,1,1
1,https://bj.lianjia.com/chengjiao/101086012217....,43436,116.453917,39.881534,7,0.051745,15,132.38,71539.0,126,2016,2004,2,High,22,1,4,6,0.667,1,1,0
2,https://bj.lianjia.com/chengjiao/101086406841....,22202,116.438010,40.076114,6,0.174618,11,134.00,51238.0,138,2016,2008,3,Bottom,21,1,1,6,0.273,1,1,0
3,https://bj.lianjia.com/chengjiao/101087277815....,52000,116.466280,39.991363,7,0.105187,14,53.00,67738.0,57,2016,2005,1,Middle,8,4,3,6,0.333,1,1,0
4,https://bj.lianjia.com/chengjiao/101087292623....,37672,116.482586,39.891991,7,0.076171,22,73.00,50112.0,167,2016,1997,2,High,6,4,4,2,0.500,0,0,0
5,https://bj.lianjia.com/chengjiao/101087508625....,55883,116.453086,39.898397,1,0.046053,10,68.00,78590.0,134,2016,2009,1,Middle,23,1,4,6,0.308,1,1,1
6,https://bj.lianjia.com/chengjiao/101087919464....,35568,116.296286,39.870107,2,0.116227,18,80.13,53588.0,90,2016,2001,1,High,24,1,4,6,0.222,1,1,1
7,https://bj.lianjia.com/chengjiao/101087921597....,60607,116.329220,39.892447,2,0.079058,29,59.40,59395.0,76,2016,1990,2,High,26,1,3,6,0.250,1,1,1
8,https://bj.lianjia.com/chengjiao/101087957433....,32981,116.284755,39.934530,8,0.126340,10,64.28,70141.0,6,2016,2009,1,Middle,19,1,3,6,0.111,1,1,1
9,https://bj.lianjia.com/chengjiao/101088008763....,30189,116.603852,39.893276,7,0.196755,11,111.30,48859.0,130,2016,2008,2,Middle,6,4,4,6,0.167,1,1,0


In [0]:
ml_dataset = df.sample(n = 25000, random_state= 5)
train_ml = ml_dataset.sample(frac= 0.8, random_state= 5)
test_ml = ml_dataset.drop(train_ml.index)

train_dl, test_dl = train_test_split(df, test_size= 0.2)

train_ml = train_ml.reset_index(drop= True)
test_ml = test_ml.reset_index(drop= True)
train_dl = train_dl.reset_index(drop= True)
test_dl = test_dl.reset_index(drop= True)

train_ml.to_csv("ML_train.csv", index= False)
test_ml.to_csv("ML_test.csv", index= False)
train_dl.to_csv("DL_train.csv", index= False)
test_dl.to_csv("DL_test.csv", index= False)

In [0]:
test_dl

,url,price,Lng,Lat,district,distance,age,square,communityAverage,followers,tradeTime,constructionTime,livingRoom,floorType,floorHeight,buildingType,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway
0,https://bj.lianjia.com/chengjiao/BJHD87280779....,1140,116.371677,40.002169,8,0.104279,26,43.86,82002.0,0,2014,1993,1,Low,6,4,1,2,0.500,0,0,0
1,https://bj.lianjia.com/chengjiao/BJHD89107759....,52110,116.382606,39.983696,8,0.083273,30,40.30,89952.0,1,2015,1989,1,Low,6,4,4,2,0.333,0,0,1
2,https://bj.lianjia.com/chengjiao/BJCP91960337....,25040,116.239192,40.230751,6,0.367327,26,70.29,38375.0,2,2016,1993,2,Low,6,4,4,2,0.500,0,0,1
3,https://bj.lianjia.com/chengjiao/BJXC84917796....,42117,116.351337,39.931197,10,0.062225,30,65.20,93676.0,0,2012,1989,2,High,16,4,1,2,0.167,1,1,1
4,https://bj.lianjia.com/chengjiao/BJXC87839354....,43698,116.343245,39.900529,10,0.064260,13,23.80,92360.0,3,2014,2006,1,High,26,1,3,6,0.111,1,1,1
5,https://bj.lianjia.com/chengjiao/BJHD00377530....,29056,116.333451,39.907225,8,0.074011,30,41.30,74064.0,0,2011,1989,1,Bottom,6,4,1,2,0.167,0,0,1
6,https://bj.lianjia.com/chengjiao/BJXC85694008....,50174,116.340073,39.899642,10,0.067481,8,92.08,101676.0,0,2013,2011,2,Low,26,3,1,6,0.500,1,1,1
7,https://bj.lianjia.com/chengjiao/BJCY88522943....,46456,116.438381,39.964399,7,0.067703,26,43.59,68475.0,29,2015,1993,1,Low,6,4,1,2,0.333,0,0,1
8,https://bj.lianjia.com/chengjiao/101101380891....,63710,116.560233,39.924347,7,0.154155,31,42.38,46480.0,129,2017,1988,1,High,6,4,3,2,0.333,0,0,0
9,https://bj.lianjia.com/chengjiao/BJFT91195878....,42625,116.370835,39.867038,2,0.052135,21,53.96,61243.0,2,2016,1998,1,Middle,22,1,4,6,0.182,1,1,0


In [0]:
df["elevator"].value_counts()

1    142284
0     89678
Name: elevator, dtype: int64

In [0]:
df.shape

(231962, 22)

In [0]:
df.isnull().sum()

url                    0
price                  0
Lng                    0
Lat                    0
district               0
distance               0
age                    0
square                 0
communityAverage       0
followers              0
tradeTime              0
constructionTime       0
livingRoom             0
floorType              0
floorHeight            0
buildingType           0
renovationCondition    0
buildingStructure      0
ladderRatio            0
elevator               0
fiveYearsProperty      0
subway                 0
dtype: int64

In [0]:
train_dl.shape

(185569, 22)